In [1]:
import pandas as pd

In [2]:
# import glob

In [3]:
# files = glob.glob("eniac-data-default/*.csv")
 
# df_list = []
# for filename in files:
#   data = pd.read_csv(filename)
#   df_list.append(data)
 
# df = pd.concat(df_list)

In [4]:
orders = pd.read_csv("eniac-data-default/orders.csv")
orderlines = pd.read_csv("eniac-data-default/orderlines.csv")
products = pd.read_csv("eniac-data-default/products.csv")
brands = pd.read_csv("eniac-data-default/brands.csv")

In [5]:
def start_pipeline(data):
    return data.copy()

In [6]:
def check_info(data):
    return data.info()

def check_nan_value(data):
    print('NaN-Values:', '\n', data.isna().sum())

def duplicates(data):
    print('Duplicated:', data.duplicated().value_counts())

def check_df(data):
    return check_info(data), duplicates(data), check_nan_value(data)

In [7]:
check_df(orders)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226909 entries, 0 to 226908
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      226909 non-null  int64  
 1   created_date  226909 non-null  object 
 2   total_paid    226904 non-null  float64
 3   state         226909 non-null  object 
dtypes: float64(1), int64(1), object(2)
memory usage: 6.9+ MB
Duplicated: False    226909
dtype: int64
NaN-Values: 
 order_id        0
created_date    0
total_paid      5
state           0
dtype: int64


(None, None, None)

## orders pipeline

In [8]:
def set_types_o(data): 
    return data.assign(created_date = pd.to_datetime(data['created_date']))

def remove_NaN_o(data):
    return data.dropna().reset_index(drop=True)

In [9]:
o_cl = (
    orders
    .pipe(start_pipeline)
    .pipe(set_types_o)
    .pipe(remove_NaN_o)
)

In [10]:
check_df(o_cl)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 226904 entries, 0 to 226903
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      226904 non-null  int64         
 1   created_date  226904 non-null  datetime64[ns]
 2   total_paid    226904 non-null  float64       
 3   state         226904 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.9+ MB
Duplicated: False    226904
dtype: int64
NaN-Values: 
 order_id        0
created_date    0
total_paid      0
state           0
dtype: int64


(None, None, None)

## orderlines pipeline

In [11]:
orderlines.sample(10)

,id,id_order,product_id,product_quantity,sku,unit_price,date
154144,1423665,432777,0,1,SNS0025,679.00,2017-11-22 23:55:47
280410,1629335,518449,0,1,ALL0022,18.99,2018-02-27 20:56:07
132544,1380814,413896,0,1,KEN0208,39.99,2017-10-18 11:50:19
7268,1138443,306215,0,1,JBL0124,89.99,2017-01-10 01:06:55
200955,1494462,462090,0,1,PAC2211,730.18,2017-12-13 18:21:19
225480,1539925,482944,0,2,OWC0256,17.09,2018-01-05 13:33:26
235095,1555905,489043,0,1,APP1216,130.00,2018-01-12 00:22:02
810,1120654,300296,0,1,WDT0374,261.99,2017-01-02 09:35:57
156450,1427638,434404,0,1,SPE0191,33.99,2017-11-23 13:17:35
58653,1236423,350147,0,1,NTE0054,22.99,2017-04-26 21:17:55


In [12]:
def rename_id_o(data):
    return data.rename(columns={'id_order':'order_id'})

def set_types_ol(data): 
    return data.assign(date = pd.to_datetime(data['date']))

def remove_dots(data):
    return data.assign(price_nd = data['unit_price'].str.replace('\.','', regex=True))

def split_price(data):
    return (data
        .assign(euro = data['price_nd'].str[:-2])
        .assign(cent = data['price_nd'].str[-2:])
    )

def merge_price(data):
    return data.assign(price_per_unit = data['euro'] + '.' + data['cent'])

def set_type_price(data): 
    return data.assign(price_per_unit = pd.to_numeric(data['price_per_unit']))

def drop_unrelevant_ol(data):
    return data.drop(columns = {'unit_price', 'price_nd', 'euro', 'cent', 'product_id', 'id'})

def rename_price(data):
    return data.rename(columns={'price_per_unit':'unit_price'})

In [13]:
ol_cl=(
orderlines
    .pipe(start_pipeline)
    .pipe(rename_id_o)
    .pipe(set_types_ol)
    .pipe(remove_dots)
    .pipe(split_price)
    .pipe(merge_price)
    .pipe(set_type_price)
    .pipe(drop_unrelevant_ol)
    .pipe(rename_price)
)


In [14]:
check_df(ol_cl)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293983 entries, 0 to 293982
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   order_id          293983 non-null  int64         
 1   product_quantity  293983 non-null  int64         
 2   sku               293983 non-null  object        
 3   date              293983 non-null  datetime64[ns]
 4   unit_price        293983 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 11.2+ MB
Duplicated: False    293983
dtype: int64
NaN-Values: 
 order_id            0
product_quantity    0
sku                 0
date                0
unit_price          0
dtype: int64


(None, None, None)

## products pipeline

In [15]:
check_df(products)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19326 entries, 0 to 19325
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sku          19326 non-null  object
 1   name         19326 non-null  object
 2   desc         19319 non-null  object
 3   price        19280 non-null  object
 4   promo_price  19326 non-null  object
 5   in_stock     19326 non-null  int64 
 6   type         19276 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.0+ MB
Duplicated: False    10580
True      8746
dtype: int64
NaN-Values: 
 sku             0
name            0
desc            7
price          46
promo_price     0
in_stock        0
type           50
dtype: int64


(None, None, None)

In [16]:
def drop_unrelevant(data):
    return data.drop(columns = {'in_stock', 'type'})

def drop_duplicates(data):
    return data.drop_duplicates().reset_index(drop=True)

def clean_prices(data):
    data.price = data.price.apply(lambda x : x +'.00' if x.count('.') == 0 else x)
    data.price = data.price.apply(lambda x: x  + '0' if x[-2]=='.' else x)
    data.price = data.price.apply(lambda x: str(float(x.replace('.',''))/1000) if ((x[-4]=='.') & (x.count('.')==2)) else x)
    data.price = data.price.apply(lambda x: str(float(x.replace('.',''))/10000) if ((x[-4]=='.') & (x.count('.')==1)) else x)
    data.price = data.price.astype(float)
    data.price = data.price.round(decimals = 2)
    return data

def clean_promo_price(data):
    data.promo_price = data.promo_price.apply(lambda x : x +'.00' if x.count('.') == 0 else x)
    data.promo_price = data.promo_price.apply(lambda x: x  + '0' if x[-2]=='.' else x)
    data.promo_price = data.promo_price.apply(lambda x: str(float(x.replace('.',''))/1000) if ((x[-4]=='.') & (x.count('.')==1)) else x)
    data.promo_price = data.promo_price.apply(lambda x: str(float(x.replace('.',''))/10000) if ((x[-4]=='.') & (x.count('.')==2)) else x)
    data.promo_price = data.promo_price.astype(float)
    data.promo_price = data.promo_price.round(decimals = 2)
    return data

def correct_promo_price(data, col1, col2): 
    for i in col1:
        for j in col2:
            if i < j: 
                data[j] = data[j].apply(lambda x: x/10)
                if i < j:
                    data[j] = data[j].apply(lambda x: x/10)
            data[j] = data[j].round(decimals = 2)
        return data


In [17]:
p_cl = (
products
    .pipe(start_pipeline)
    .pipe(remove_NaN_o)
    .pipe(drop_unrelevant)
    .pipe(drop_duplicates)
    .pipe(clean_prices)
    .pipe(clean_promo_price)
#     .pipe(correct_promo_price, ['price'], ['promo_price'])
)

In [18]:
p_cl.iloc[1488]

sku                                                     NTE0082
name                NewerTech NuCube stand upright for Mac Mini
desc           Protective stand for Mac Mini from 2010 to 2014.
price                                                     60.99
promo_price                                               44.99
Name: 1488, dtype: object

#### check if order_id from ol_cl in o_cl

In [19]:
(
ol_cl
    .assign(check_orders = ol_cl['order_id'].isin(o_cl['order_id']))
)['check_orders'].sum()

293743

In [20]:
(
ol_cl
    .assign(check_orders = ol_cl['order_id'].isin(o_cl['order_id']))
    .query("check_orders==False")
)

,order_id,product_quantity,sku,date,unit_price,check_orders
5,295310,10,WDT0249,2017-01-01 01:14:27,231.79,False
63,296284,1,BNQ0042,2017-01-01 09:58:35,699.00,False
67,294806,1,APP1849,2017-01-01 10:09:15,2558.99,False
69,294806,1,APP1864,2017-01-01 10:15:14,2797.99,False
70,297261,1,QNA0177,2017-01-01 10:17:59,304.99,False
...,...,...,...,...,...,...
275665,244328,1,OWC0260,2018-02-20 13:00:45,349.00,False
280055,261391,1,APP2352,2018-02-27 11:43:25,3343.00,False
280352,287797,1,SYN0182,2018-02-27 19:16:26,484.11,False
280856,261391,1,SAT0091,2018-02-28 12:18:26,44.99,False


#### correction

In [21]:
ol_cl = (
ol_cl
    .assign(check_orders = ol_cl['order_id'].isin(o_cl['order_id']))
    .query("check_orders==True")
)

In [22]:
ol_cl = ol_cl.drop(columns = {"check_orders"})

#### check if order_id from o_cl in ol_cl

In [23]:
(
o_cl
    .assign(check_orders = o_cl['order_id'].isin(ol_cl['order_id']))
)['check_orders'].sum()

204691

In [24]:
(
o_cl
    .assign(check_orders = o_cl['order_id'].isin(ol_cl['order_id']))
    .query("check_orders==False")
)

,order_id,created_date,total_paid,state,check_orders
8,245941,2017-01-01 10:32:23,183.52,Completed,False
65,268629,2017-01-31 11:27:25,73.23,Completed,False
75,272862,2017-04-18 18:17:10,128.99,Pending,False
82,277655,2017-02-01 08:44:55,9.99,Pending,False
83,277994,2017-01-23 18:30:11,52.99,Completed,False
...,...,...,...,...,...
226830,527328,2018-03-14 13:14:12,0.00,Shopping Basket,False
226846,527344,2018-03-14 13:25:21,0.00,Place Order,False
226848,527346,2018-03-14 13:26:14,0.00,Place Order,False
226850,527348,2018-03-14 13:28:18,0.00,Place Order,False


#### correction

In [25]:
o_cl = (
o_cl
    .assign(check_orders = o_cl['order_id'].isin(ol_cl['order_id']))
    .query("check_orders==True")
)

In [26]:
o_cl = o_cl.drop(columns = {"check_orders"})

#### check if sku from ol_cl in p_cl

In [27]:
(
ol_cl
    .assign(check_sku = ol_cl['sku'].isin(p_cl['sku']))
)['check_sku'].sum()

292008

In [28]:
(
ol_cl
    .assign(check_sku = ol_cl['sku'].isin(p_cl['sku']))
    .query("check_sku==False")
)

,order_id,product_quantity,sku,date,unit_price,check_sku
117,299638,1,SYN0127,2017-01-01 11:56:18,223.24,False
192,299706,1,EVU0007,2017-01-01 13:57:16,28.49,False
198,299712,1,APP0608,2017-01-01 14:10:47,279.99,False
253,299766,1,APP0607,2017-01-01 16:09:02,279.99,False
285,299799,1,APP0608,2017-01-01 17:09:18,279.99,False
...,...,...,...,...,...,...
283903,521165,1,CEL0044,2018-03-05 07:35:33,7.99,False
285011,521667,1,CEL0044,2018-03-05 13:39:35,7.99,False
291946,525658,1,DOD0010,2018-03-11 22:37:54,21.99,False
291957,525670,1,DOD0010,2018-03-11 22:54:41,21.99,False


#### correction:

In [29]:
ol_cl = (
ol_cl
    .assign(check_sku = ol_cl['sku'].isin(p_cl['sku']))
    .query("check_sku==True")
)

#### check if sku from p_cl in ol_cl

In [30]:
(
p_cl
    .assign(check_sku = p_cl['sku'].isin(ol_cl['sku']))
)['check_sku'].sum()

7807

In [31]:
(
p_cl
    .assign(check_sku = p_cl['sku'].isin(ol_cl['sku']))
    .query("check_sku==False")
)

,sku,name,desc,price,promo_price,check_sku
6,KIN0008,Mac Memory Kingston 1GB 667MHz DDR2 SO-DIMM,1GB RAM Mac mini and iMac (2006/07) MacBook Pr...,18.99,146.47,False
11,SEN0021,Sennheiser CX 300-II Precision headphones iPho...,Headphones iPhone iPad iPad 2 iPad 3 and iPod.,49.99,449.88,False
21,APP0233,Apple iPad Camera Connection connector,IPad connector for digital cameras and SD cards.,35.00,349.90,False
22,JMO0014,Just Mobile Upstand iPad support,Support Retina iPad and iPad mini aluminum.,49.95,399.90,False
23,MOB0007,Mobee Magic Charger Charger Magic Mouse,Wireless Charger for Magic Mouse,49.99,499.85,False
...,...,...,...,...,...,...
10472,BEL0376,Belkin Travel Support Apple Watch Black,compact and portable stand vertically or horiz...,29.99,269.90,False
10473,THU0060,"Enroute Thule 14L Backpack MacBook 13 ""Black",Backpack with capacity of 14 liter compartment...,69.95,649.90,False
10474,THU0061,"Enroute Thule 14L Backpack MacBook 13 ""Blue",Backpack with capacity of 14 liter compartment...,69.95,649.90,False
10475,THU0062,"Enroute Thule 14L Backpack MacBook 13 ""Red",Backpack with capacity of 14 liter compartment...,69.95,649.90,False


#### correction

In [32]:
p_cl = (
p_cl
    .assign(check_sku = p_cl['sku'].isin(ol_cl['sku']))
    .query("check_sku==True")
)

In [33]:
p_cl = p_cl.drop(columns = {"check_sku"})

#### checking samples

In [34]:
check_df(p_cl)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7807 entries, 0 to 10471
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          7807 non-null   object 
 1   name         7807 non-null   object 
 2   desc         7807 non-null   object 
 3   price        7807 non-null   float64
 4   promo_price  7807 non-null   float64
dtypes: float64(2), object(3)
memory usage: 366.0+ KB
Duplicated: False    7807
dtype: int64
NaN-Values: 
 sku            0
name           0
desc           0
price          0
promo_price    0
dtype: int64


(None, None, None)

In [35]:
p_cl.sample(10)

,sku,name,desc,price,promo_price
4842,IFX0141,iFixit Destornilador Tri-point Y00 A1278 Macbo...,MacBook Trackpad screwdriver 2008 / MacBook Pr...,4.99,44.89
5877,APP1914,"Apple MacBook Pro 15 ""Core i7 Touch Bar 27GHz ...",New MacBook Pro 15-inch Touch Bar to 27GHz Cor...,3679.00,3285.59
5256,OWC0097-A,Open - OWC Thunderbolt Cable 1m Black,Thunderbolt Cable 1 meter,48.99,359.90
4693,HTE0009,Hyper Pearl 3000mAh USB Battery mirror and Silver,Mirror + Portable 3000mAh USB (2.4A) battery c...,34.95,299.90
1518,TWS0085,Twelve South HiRise Deluxe iPhone and iPad Black,Adjustable Stand for iPhone 5 Lightning / 5C /...,69.99,569.90
7758,APP2084,"Apple MacBook Pro 13 ""Core i5 with Touch Bar 3...",New MacBook Pro 13 inch Touch Bar 31 GHz Core ...,2005.59,1885.00
5631,MUJ0018,Mujjo iPhone Leather Case Leather Champagne 8/7/6,ultrathin Case for iPhone vegetable tanned lea...,39.90,359.90
10116,CAV0010,Cavus Foot Support Sonos Play 1 White,Floor stand for Speaker Sonos Play 1,59.00,71.39
2517,IHE0016,iHealth Ismart Alarm Command for home,Command to activate and deactivate alarm range...,24.95,249.86
6601,EIZ0024,"Eizo FlexScan EV2736W Monitor 27 ""QHD DP pivot...",27-inch monitor with slim frame and adjustable...,629.00,649.99


## merging o_cl with ol_cl

In [36]:
orders_info = (
o_cl
    .merge(ol_cl, how='inner', left_on='order_id', right_on='order_id').reset_index()
    .copy()
)

In [41]:
check_df(o_cl)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204691 entries, 0 to 226903
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      204691 non-null  int64         
 1   created_date  204691 non-null  datetime64[ns]
 2   total_paid    204691 non-null  float64       
 3   state         204691 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 7.8+ MB
Duplicated: False    204691
dtype: int64
NaN-Values: 
 order_id        0
created_date    0
total_paid      0
state           0
dtype: int64


(None, None, None)

## finding NaN-Values

In [38]:
def find_nan_rows(data, column):
    for val in column:
        data = data[data[val].isna()]
    return data


In [39]:
check_df(p_cl)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7807 entries, 0 to 10471
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   sku          7807 non-null   object 
 1   name         7807 non-null   object 
 2   desc         7807 non-null   object 
 3   price        7807 non-null   float64
 4   promo_price  7807 non-null   float64
dtypes: float64(2), object(3)
memory usage: 366.0+ KB
Duplicated: False    7807
dtype: int64
NaN-Values: 
 sku            0
name           0
desc           0
price          0
promo_price    0
dtype: int64


(None, None, None)

In [40]:
# p_cl.to_csv("eniac-data-clean/products_cl.csv", index=False)
# orders_info.to_csv("eniac-data-clean/orders_info_cl.csv", index=False)
# ol_cl.to_csv("eniac-data-clean/orderlines_cl.csv", index=False)
# o_cl.to_csv("eniac-data-clean/orders_cl.csv", index=False)